# Face Recognizer

In [98]:
# Assuming for collecting the dataset of images directories are already created under test and train directory as n0,n1,n1....
# acoording to the data of number of persons

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

def capture_images(t,name):
    # Initialize Webcam
    cap = cv2.VideoCapture(0)
    count = 1

    # Collect 100 samples of your face from webcam input
    while True:

        ret, frame = cap.read()
        if face_extractor(frame) is not None:
            count += 1
            face = cv2.resize(face_extractor(frame), (200, 200))
            #face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Save file in specified directory with unique name
            file_name_path = t+"image" + str(count) + '.jpg'
            cv2.imwrite(file_name_path, face)

            # Put count on images and display live count
            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow(name, face)

        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
            break

    cap.release()
    cv2.destroyAllWindows()      
    return "Collecting Samples Complete"

### Collecting Dataset

In [99]:
import cv2
import numpy as np
import time

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

file_path='C://Users//500060783//Desktop//MLOPS-WS//Projects//Face Recognizer Transfer Learning//Family//'
train_path=file_path+"train//"
test_path=file_path+"test//"
c=int(input("Enter the number of persons you want to capture image dataset"))
for i in range(c):
    print(f"Provide your images for testing dataset for person {i+1}")
    time.sleep(2)
    t=train_path+"n"+str(i)+"//"
    name="train images"
    out=capture_images(t,name)+f" for train set of person{i+1}"
    time.sleep(1)
    print(out)
    time.sleep(2)
    t=test_path+"n"+str(i)+"//"
    name="test images"
    out=capture_images(t,name)+f" for test set of person{i+1}"
    time.sleep(1)
    print(out)
    time.sleep(2)
print("Data Set completed successfully")

Enter the number of persons you want to capture image dataset1
Provide your images for testing dataset for person 1
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Data Set completed successfully


### Loading the VGG-16 Model

In [1]:
from keras.applications import vgg16

# VGG-16 was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the VGG-16 model without the top or FC layers
Vgg16 = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in Vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(Vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.


58892288/58889256 [==============================] - 23s 0us/step
0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


### Let's make a function that returns our FC Head

In [3]:
def add_fc(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

### Let's add our FC Head back onto VGG-16

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 3

FC_Head = add_fc(Vgg16, num_classes)

model = Model(inputs = Vgg16.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

### Loading our FaceDataset

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Family/train'
validation_data_dir = 'Family/test'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 605 images belonging to 3 classes.
Found 255 images belonging to 3 classes.


### Training out Model
- Note we're using checkpointing and early stopping

In [7]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("face_recognize_vgg16.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 605
nb_validation_samples = 255

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
37/37 [==============================] - 689s 19s/step - loss: 1.1333 - accuracy: 0.4847 - val_loss: 0.4888 - val_accuracy: 0.7286

Epoch 00001: val_loss improved from inf to 0.48878, saving model to face_recognize_vgg16.h5
Epoch 2/5
37/37 [==============================] - 704s 19s/step - loss: 0.5636 - accuracy: 0.7716 - val_loss: 0.8072 - val_accuracy: 0.6360

Epoch 00002: val_loss did not improve from 0.48878
Epoch 3/5
37/37 [==============================] - 658s 18s/step - loss: 0.4762 - accuracy: 0.8260 - val_loss: 0.3126 - val_accuracy: 0.7699

Epoch 00003: val_loss improved from 0.48878 to 0.31258, saving model to face_recognize_vgg16.h5
Epoch 4/5
37/37 [==============================] - 662s 18s/step - loss: 0.3578 - accuracy: 0.8744 - val_loss: 0.7186 - val_accuracy: 0.6987

Epoch 00004: val_loss did not improve from 0.31258
Epoch 5/5
37/37 [==============================] - 646s 17s/step - loss: 0.3158 - accuracy: 0.8908 - val_loss: 1.2518 - val_accuracy: 0.6276



### Loading our classifer

In [8]:
from keras.models import load_model

classifier = load_model('face_recognize_vgg16.h5')

### Testing our classifer on some test images

In [103]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

face_recognize_dict = {"[0]": "Saurabh ", 
                      "[1]": "Piyush",
                      "[2]": "Aditya"}

face_recognize_dict_n = {"n0": "Saurabh ", 
                      "n1": "Piyush",
                      "n2": "Aditya"}

def draw_test(name, pred, im):
    face = face_recognize_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + face_recognize_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("Family/test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Piyush
Class - Saurabh 
Class - Piyush
Class - Saurabh 
Class - Saurabh 
Class - Piyush
Class - Saurabh 
Class - Saurabh 
Class - Piyush
Class - Aditya


### Prediction on LiveStream

In [101]:
import cv2
cap=cv2.VideoCapture(0)
while True:
    status,photo=cap.read()
    input_im=photo
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original)
    
    if cv2.waitKey(1)==13:
        break
cv2.destroyAllWindows()
cap.release()